In [1]:
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [2]:
from importlib import reload
import pickle

In [3]:
with open("bsDoctor-report/data.pickle", 'rb') as fd:
    data = pickle.load(fd)

In [16]:
import bsDoctor
reload(bsDoctor)
import src.utils
reload(src.utils)

<module 'src.utils' from '/root/bsDoctor/src/utils.py'>

In [5]:
from jinja2 import Environment,FileSystemLoader

In [18]:
env = Environment(loader=FileSystemLoader('report/'))
env.globals["include_file"] = src.utils.include_file
# template = env.get_template('base.html')   
template = env.get_template('main.jinja-html')
temp_out = template.render(alldata=data)   

with open('bsDoctor-report/sample-2.html', 'w', encoding='utf-8') as f:
    f.writelines(temp_out)

In [15]:
data

{'SAMPLE': 'ENCFF873NOV',
 'title': 'bsDoctor Report of Sample: ENCFF873NOV',
 'bin_size': '100 Kbp',
 'lambda_DNA_content_vs_nuclear': '0.2145',
 'lambda_DNA_content_vs_MT': '38',
 'mt_is_covered': 1,
 'mt_nCorG': '7,350',
 'mt_covnCorG': '7,133',
 'mt_covnCorG_prop': '97.05',
 'mt_length': '16,568',
 'mt_covn': '16,568',
 'mt_cov_prop': '100',
 'mt_bin_size': '20',
 'mt_median_dp': '4.195e+04',
 'mt_mean_dp': '4.595e+04',
 'mt_me': '0.009847',
 'bsrate_mt': '99.02',
 'err_rate_mt': '7.03',
 'lambda_is_covered': 1,
 'lambda_nCorG': '24,182',
 'lambda_covnCorG': '24,182',
 'lambda_covnCorG_prop': '100',
 'lambda_length': '48,502',
 'lambda_covn': '48,502',
 'lambda_cov_prop': '100',
 'lambda_size': '50',
 'lambda_median_dp': '3685',
 'lambda_mean_dp': '5947',
 'bsrate_lambda': '99.01',
 'err_rate_lambda': '15.61',
 'nCHH': '2,226,287',
 'bsrate_chh': '98.99',
 'bs_rate': '99.01',
 'me_CG': '51.93',
 'me_CG_adj': '51.45',
 'me_CHG': '1.137',
 'me_CHG_adj': '0.1491',
 'me_CHH': '1.099',
